In [1]:
#
# Importing all necessary libraries. We shall use ICP registration and other functions
# (such mesh handler, visualization) provided in Open3D. Also some speed-ups with JIT.
#
import open3d as o3d
import numpy as np
import scipy.optimize as opt
import multiprocess as mp
import matplotlib.pyplot as plt
import csv
import copy
from numba import jit

In [2]:
#
# Number of CPU cores
#
CPU_COUNT = mp.cpu_count()

In [3]:
#
# Barycenter of a point cloud A
#
@jit(target_backend='cuda')
def bar(A):
    #
    return np.sum(A, axis=1)/A.shape[1]

In [4]:
#
# Centering a point cloud A
#
@jit(target_backend='cuda')
def barycentered(A):
    #
    bar_A = bar(A)
    #
    return np.array([vec - bar_A for vec in A.T]).T

In [5]:
#
# First constructing a random psd matrix with specified condition number,
# then multiplying it by a random orthogonal matrix 
#
# Bierlaire, M., Toint, P., and Tuyttens, D. (1991). 
# On iterative algorithms for linear ls problems with bound constraints. 
# Linear Algebra and Its Applications, 143, 111–143.
#
@jit(target_backend='cuda')
def rand_mat_cond(dim=3, cond=5.0):
    #
    log_cond = np.log(cond)
    exp_vec = np.arange(-log_cond/4.0, log_cond * (dim + 1)/(4 * (dim - 1)),\
                                                                    log_cond/(2.0*(dim-1)))
    D = np.diag(np.exp(exp_vec[:dim]))
    U, _ = np.linalg.qr((np.random.rand(dim,dim) - 5.0) * 200)
    V, _ = np.linalg.qr((np.random.rand(dim,dim) - 5.0) * 200)
    P = U @ D @ V.T
    P = P @ P.T
    #
    seed = np.random.normal(0.0, 1.0, (dim,dim))
    O, _ = np.linalg.qr(seed, mode='complete')
    if np.linalg.det(O) < 0:
        O = O @ np.diag([1]*(dim-1)+[-1])
    #
    M = P @ O
    #
    return M

In [6]:
#
# Orthogonal projection onto ker X: R^n -> R^d (generically dim ker X = n-d)
#
@jit(target_backend='cuda')
def ortho_proj_ker(X):
    #
    Ux, Sigmax, Vx = np.linalg.svd(X)
    rank = Sigmax.shape[0]
    ker = Vx[:rank]
    proj = ker.T @ ker
    #
    return proj

In [7]:
#
# PSD matrix mapping the ellipsoid of X to the ellipsoid of Y
#
# Let Ex = X @ X.T, Ey = Y @ Y.T, then it return P, a symmetric psd matrix, 
# such that P @ Ex @ P.T = Ey. Such P is uniquely determined by Ex and Ey. 
#
def psd_fit(X, Y):
    #
    Ux, Sigma_x, _ = np.linalg.svd(X)
    Uy, Sigma_y, _ = np.linalg.svd(Y)
    #
    Lx = Ux @ np.diag(Sigma_x)
    Ly = Uy @ np.diag(Sigma_y)
    #
    Lxy = Lx.T @ Ly
    #
    W, Sigma, V = np.linalg.svd(Lxy)
    P = W @ np.diag(Sigma) @ W.T   # the PSD part P is uniquely determined by X and Y
    # O = W @ V                    # while the orthogonal part is not (we do not need it)
    Lx_inv = np.linalg.inv(Lx)
    P = Lx_inv.T @ P @ Lx_inv
    #
    return P

In [8]:
#
# Input: meshes mesh_1, mesh_2; sample sizes size_1, size_2; condition number cond=3.0; 
# number of iterations of rFAQ n_iter=2**10; verbose = (True|False)
#
# Assumption: cardinality of X <= cardinality of Y
#
# Algorithm: adjust the barycenter of meshes to 0. Sample size_1 points from mesh_1, i.e. create point cloud X; apply a random 
# linear transform L (having cond(L) = cond) to mesh_2 and then sample size_2 points from 
# it, i.e. create point cloud Y. Run GraNNI on the point clouds X and Y, with number of
# rFAQ iterations equal n_iter: obtain the recovered linear map L_0. 
#
# Output: L and L_0
#
def granni_point_clouds(mesh_1, mesh_2, size_1, size_2, cond=3.0, n_iter=2**10, verbose=False):
    #
    dim = 3
    #
    temp_mesh_1 = copy.deepcopy(mesh_1)
    temp_mesh_2 = copy.deepcopy(mesh_2)
    #
    L = rand_mat_cond(dim, cond)
    transformation = np.c_[np.c_[L, [0,0,0]].T, [0,0,0,1]].T
    temp_mesh_2.transform(transformation)
    #    
    pcl_mesh_1 = temp_mesh_1.sample_points_poisson_disk(number_of_points=size_1)
    pcl_mesh_2 = temp_mesh_2.sample_points_poisson_disk(number_of_points=size_2)
    #
    X = np.asarray(pcl_mesh_1.points).T
    Y = np.asarray(pcl_mesh_2.points).T
    #
    num_x = X.shape[1]
    num_y = Y.shape[1]
    #
    assert(dim == X.shape[0] == 3)
    assert(dim == Y.shape[0] == 3)
    assert(num_x <= num_y)
    #
    if verbose:
        print("Number of points in sample 1:   {}".format(num_x))
        print("Number of points in sample 2:   {}".format(num_y))
    #
    X = barycentered(X)
    Y = barycentered(Y)
    #
    Px = ortho_proj_ker(X)
    o  = np.zeros((num_y-num_x, num_y-num_x))
    oo = np.zeros((num_y-num_x, num_x))
    Px = np.block([[Px, oo.T],[oo, o]])
    Py = ortho_proj_ker(Y)
    I = np.identity(num_y)
    #
    def f_opt(i):
        np.random.seed(i) # reseed to avoid races for the RNG
        sol = opt.quadratic_assignment(Px, Py, method="faq", options = {'maximize':True,\
                                                            'P0':'randomized', 'tol':1e-3})
        ind = sol['col_ind']
        s = I[ind]
        weight = np.exp(-1000.0*(np.trace(Px @ s @ Py @ s.T) - dim)**2)
        return weight, s, i
    #
    with mp.Pool(CPU_COUNT, maxtasksperchild=int(100)) as pool:
        vals = pool.map(f_opt, range(n_iter))
    #    
    mat = sum([v[0]*v[1] for v in vals])
    #
    row_ind, col_ind = opt.linear_sum_assignment(mat, maximize=True)
    #
    L0 = Y[:, col_ind][:, :num_x] @ X.T @ np.linalg.inv(X @ X.T)
    #
    return L, L0

In [9]:
#
# Testing GraNNI on two meshes source_mesh and target_mesh; samples have sizes 
# source_size and target_size, respectively. Linear transform for the purpose of
# testing has condition number cond (default 3.0); the number of calls to rFAQ 
# in GraNNI equals n_iter (default 2**10).
#
# Output: the preimage of target_mesh compared to source_mesh.
#
def granni_test(source_mesh, target_mesh, source_size, target_size, cond=3.0, n_iter=2**10):
    #
    mesh_1 = copy.deepcopy(source_mesh)
    size_1 = source_size
    mesh_2 = copy.deepcopy(target_mesh)
    size_2 = target_size
    #
    c_mesh_1 = mesh_1.get_center()
    mesh_1.translate(-1.0*c_mesh_1)
    assert np.allclose(mesh_1.get_center(), np.zeros((1,3)))
    #
    c_mesh_2 = mesh_2.get_center()
    mesh_2.translate(-1.0*c_mesh_2)
    assert np.allclose(mesh_2.get_center(), np.zeros((1,3)))
    #
    L, L0 =\
    granni_point_clouds(mesh_1, mesh_2, size_1, size_2, cond=cond, n_iter=n_iter, verbose=True)
    #
    transform_mesh_1 = copy.deepcopy(mesh_1)
    transform_mesh_2 = copy.deepcopy(mesh_2)
    transformation = np.linalg.inv(L0) @ L
    print("Original transform L :")
    print(L)
    print("Recovered L_0 :")
    print(L0)
    print("Product L^{-1}_0 @ L :")
    print(transformation)
    print("Relative difference |L - L_0|/|L| :")
    norm_L = np.linalg.norm(L, 2)
    diff = np.linalg.norm(L - L0, 2)/norm_L
    print(diff)
    transformation = np.c_[np.c_[transformation, [0,0,0]].T, [0,0,0,1]].T
    #
    transform_mesh_1.compute_vertex_normals()
    transform_mesh_1.paint_uniform_color([1, 0.706, 0])
    transform_mesh_2.compute_vertex_normals()
    transform_mesh_2.paint_uniform_color([0, 0.651, 0.929])
    transform_mesh_2.transform(transformation)
    transform_mesh_2.translate(- np.linalg.inv(L0) @ L @ c_mesh_1 + c_mesh_2)
    #
    o3d.visualization.draw([transform_mesh_1, transform_mesh_2])

In [10]:
#
# Loading the "Enfant au Chien" statue: before restoration (broken) 
# and after (major defects repaired and gaps filled in)
#
mesh_Enfant_1 = o3d.io.read_triangle_mesh('Enfant_au_chien-Broken.obj')
#
mesh_Enfant_2 = o3d.io.read_triangle_mesh('Enfant_au_chien.obj')

In [11]:
#
# Testing GraNNI with the given parameters.
#
granni_test(mesh_Enfant_1, mesh_Enfant_2, 600, 600, cond=5.0, n_iter=2**10)

Number of points in sample 1:   600
Number of points in sample 2:   600
Original transform L :
[[-1.06833822  0.54055008 -0.07788199]
 [-0.76678042 -0.81921115 -0.31775538]
 [ 1.48922718  0.14184523  1.07816891]]
Recovered L_0 :
[[-1.0063721   0.51029133 -0.06031949]
 [-0.70441382 -0.8458755  -0.32050824]
 [ 1.34514733  0.15507388  1.08424727]]
Product L^{-1}_0 @ L :
[[ 1.05906792 -0.03916898  0.01528543]
 [ 0.00206862  0.98657865 -0.00705641]
 [ 0.05930751  0.03831295  0.97643965]]
Relative difference |L - L_0|/|L| :
0.07688516407108238
FEngine (64 bits) created at 0x1141d0000 (threading is enabled)
FEngine resolved backend: OpenGL


[error] GLFW error: Cocoa: Failed to find service port for display
